# Premier League V4.5: Re-Optimizing for Draws

A accuracy baixou porque mudámos as regras do jogo (pesos) mas mantivemos a estratégia antiga.
Nesta etapa, vamos correr o **Grid Search** novamente, mas desta vez informando o Grid Search de que os empates são importantes.

Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib # Para salvar o modelo
import re
import os
import codecs
import requests
from bs4 import BeautifulSoup
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## 1. Data Acquisition (Recolha de Dados)
Vamos buscar dados reais do `football-data.co.uk`. Vamos carregar várias temporadas consecutivas para que o modelo tenha histórico suficiente para aprender padrões.

* **FTHG**: Full Time Home Goals
* **FTAG**: Full Time Away Goals
* **FTR**: Full Time Result (H=Home, D=Draw, A=Away)

In [ ]:
# --- CONFIGURAÇÃO ---
DATA_FILE = 'premier_league_full.csv'
XG_FILE = 'premier_league_xg_data.csv'
START_YEAR = 2005
END_YEAR = 2025

# --- FUNÇÃO 1: Scraper Robusto (Understat) ---
def scrape_understat_season(year):
    print(f"🕷️ A recolher xG de {year}/{year+1}...")
    url = f"https://understat.com/league/EPL/{year}"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return pd.DataFrame()
        
        match = re.search(r"datesData\s*=\s*JSON\.parse\('(.*?)'\)", response.text)
        if not match:
            print(f"⚠️ Sem dados para {year}")
            return pd.DataFrame()
            
        json_data = codecs.decode(match.group(1), 'unicode_escape')
        data = json.loads(json_data)
        
        matches = []
        for m in data:
            if m['isResult']:
                matches.append({
                    'Date': m['datetime'][:10],
                    'HomeTeam': m['h']['title'],
                    'AwayTeam': m['a']['title'],
                    'Home_xG': float(m['xG']['h']),
                    'Away_xG': float(m['xG']['a'])
                })
        return pd.DataFrame(matches)
    except Exception as e:
        print(f"⚠️ Erro no ano {year}: {e}")
        return pd.DataFrame()

# --- FUNÇÃO 2: Carregar Dados Principais (Football-Data) ---
def get_main_data(start, end):
    if os.path.exists(DATA_FILE):
        print(f"📂 Carregando dados locais: {DATA_FILE}")
        df = pd.read_csv(DATA_FILE)
        # Importante: Não converter data aqui ainda para controlar formato no main
        return df
    
    print("🌐 A descarregar dados do Football-Data...")
    dfs = []
    base_url = "https://www.football-data.co.uk/mmz4281/{}/{}.csv"
    for year in range(start, end + 1):
        season = f"{str(year)[-2:]}{str(year+1)[-2:]}"
        try:
            df = pd.read_csv(base_url.format(season, "E0"))
            # Forçar conversão imediata para evitar problemas de mistura
            df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
            dfs.append(df)
        except: pass
        
    full_df = pd.concat(dfs, ignore_index=True).dropna(subset=['Date', 'FTR'])
    full_df.to_csv(DATA_FILE, index=False)
    return full_df.sort_values('Date').reset_index(drop=True)

# --- FUNÇÃO 3: Limpeza de Nomes ---
def clean_team_name(name):
    name_map = {
        'Manchester United': 'Man United', 'Manchester City': 'Man City',
        'Newcastle United': 'Newcastle', 'West Ham United': 'West Ham', 'West Ham': 'West Ham',
        'Wolverhampton Wanderers': 'Wolves', 'Brighton': 'Brighton',
        'Leicester City': 'Leicester', 'Leeds United': 'Leeds',
        'Tottenham Hotspur': 'Tottenham', 'Tottenham': 'Tottenham', 
        'Nottingham Forest': "Nott'm Forest", 'Sheffield United': 'Sheffield United', 
        'Luton': 'Luton', 'Brentford': 'Brentford', 'Bournemouth': 'Bournemouth',
        'Ipswich Town': 'Ipswich', 'Hull City': 'Hull', 'Stoke City': 'Stoke',
        'Swansea City': 'Swansea', 'Cardiff City': 'Cardiff',
        'Huddersfield Town': 'Huddersfield', 'West Bromwich Albion': 'West Brom',
        'Norwich City': 'Norwich', 'Queens Park Rangers': 'QPR'
    }
    return name_map.get(name, name)

# ==========================================
# 🚀 EXECUÇÃO E LIMPEZA (A PARTE CRÍTICA)
# ==========================================

# 1. Carregar Dados Principais
df = get_main_data(START_YEAR, END_YEAR)

# Limpeza de Datas e Duplicados no Dataset Principal
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df = df.dropna(subset=['Date'])
# Remove duplicados exatos no ficheiro principal
df = df.drop_duplicates(subset=['Date', 'HomeTeam', 'AwayTeam'])

# 2. Carregar ou Sacar xG
if os.path.exists(XG_FILE):
    print("📂 Carregando xG local...")
    df_xg = pd.read_csv(XG_FILE)
else:
    print("🌐 A iniciar scraping xG...")
    dfs_xg = [scrape_understat_season(y) for y in range(START_YEAR, END_YEAR)]
    df_xg = pd.concat(dfs_xg, ignore_index=True)
    df_xg['HomeTeam'] = df_xg['HomeTeam'].apply(clean_team_name)
    df_xg['AwayTeam'] = df_xg['AwayTeam'].apply(clean_team_name)
    df_xg.to_csv(XG_FILE, index=False)

# 3. PREPARAÇÃO PARA MERGE
df_xg['Date'] = pd.to_datetime(df_xg['Date']).dt.normalize()
df['Date'] = df['Date'].dt.normalize()

# --- CORREÇÃO: Remover duplicados no xG ANTES do Merge ---
print(f"📊 Linhas xG antes da limpeza: {len(df_xg)}")
df_xg = df_xg.drop_duplicates(subset=['Date', 'HomeTeam', 'AwayTeam'], keep='first')
print(f"📉 Linhas xG limpas: {len(df_xg)}")

# Aplicar limpeza de nomes
df['HomeTeam'] = df['HomeTeam'].apply(clean_team_name)
df['AwayTeam'] = df['AwayTeam'].apply(clean_team_name)
df_xg['HomeTeam'] = df_xg['HomeTeam'].apply(clean_team_name)
df_xg['AwayTeam'] = df_xg['AwayTeam'].apply(clean_team_name)

# Remover colunas antigas de xG no DF principal
cols_exclude = [c for c in df.columns if 'xG' in c]
df_clean = df.drop(columns=cols_exclude)

# 4. MERGE FINAL
print("🔄 A realizar o Merge...")
df_final = df_clean.merge(
    df_xg[['Date', 'HomeTeam', 'AwayTeam', 'Home_xG', 'Away_xG']],
    on=['Date', 'HomeTeam', 'AwayTeam'],
    how='left'
)

# 5. REMOVER FUTURO (Segurança contra erro de datas)
hoje = pd.Timestamp.now().normalize()
antes = len(df_final)
df_final = df_final[df_final['Date'] <= hoje]
print(f"📅 Jogos removidos (futuro/datas erradas): {antes - len(df_final)}")

# Ordenar Cronologicamente
df = df_final.sort_values(['Date', 'HomeTeam', 'AwayTeam']).reset_index(drop=True)

# Estatística
missing_count = df['Home_xG'].isna().sum()
print(f"✅ Merge concluído! Jogos com xG: {len(df) - missing_count} / {len(df)}")
print(f"📉 Jogos sem xG (Preenchidos com 1.0): {missing_count}")

# Preencher vazios
df = df.fillna({'Home_xG': 1.0, 'Away_xG': 1.0})

print("🔍 A verificar duplicados no final:")
display(df.tail(3))

## 2. Feature Engineering Completa (ELO + Stats + Odds)

Aqui adicionamos as colunas B365H, B365D, B365A (Odds da Bet365).

In [ ]:
def prepare_features_corrected(df, window=5):
    print(f"📊 INICIO: {len(df)} jogos recebidos.")
    df = df.copy()
    
    # 1. LIMPEZA DE DATAS E ORDEM
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    hoje = pd.Timestamp.now().normalize()
    
    # Remover jogos do futuro
    df_clean = df[df['Date'] <= hoje].copy()
    print(f"📉 Pós-filtro de data (até hoje): {len(df_clean)} jogos.")
    
    # Ordenar cronologicamente (CRUCIAL para o shift funcionar bem)
    df_clean = df_clean.sort_values(['Date', 'HomeTeam', 'AwayTeam']).reset_index(drop=True)

    # 2. ELO CALCULATION
    elo_dict = {}
    df_clean['HomeElo'] = 1500.0
    df_clean['AwayElo'] = 1500.0
    k_factor = 20
    
    for i, row in df_clean.iterrows():
        h, a, res = row['HomeTeam'], row['AwayTeam'], row['FTR']
        h_elo = elo_dict.get(h, 1500.0)
        a_elo = elo_dict.get(a, 1500.0)
        
        df_clean.at[i, 'HomeElo'] = h_elo
        df_clean.at[i, 'AwayElo'] = a_elo
        
        if res == 'H': val = 1
        elif res == 'D': val = 0.5
        else: val = 0
        
        exp_h = 1 / (1 + 10**((a_elo - h_elo)/400))
        new_h = h_elo + k_factor * (val - exp_h)
        new_a = a_elo + k_factor * ((1-val) - (1-exp_h))
        
        elo_dict[h] = new_h
        elo_dict[a] = new_a
        
    df_clean['EloDiff'] = df_clean['HomeElo'] - df_clean['AwayElo']
    
    # 3. ROLLING STATS
    # Preencher NaNs nas colunas originais antes de calcular médias
    cols_origin = ['FTHG', 'FTAG', 'HS', 'HST', 'HC', 'Home_xG', 'Away_xG']
    for col in cols_origin:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna(0)

    # Criar auxiliares de pontos (serão removidas no fim)
    df_clean['Home_Points_Actual'] = df_clean['FTR'].map({'H':3, 'D':1, 'A':0})
    df_clean['Away_Points_Actual'] = df_clean['FTR'].map({'A':3, 'D':1, 'H':0})
    
    # Preparar DataFrame unificado para calcular médias
    cols_home = ['Date', 'HomeTeam', 'FTHG', 'FTAG', 'HS', 'HST', 'HC', 'Home_xG', 'Home_Points_Actual']
    cols_away = ['Date', 'AwayTeam', 'FTAG', 'FTHG', 'AS', 'AST', 'AC', 'Away_xG', 'Away_Points_Actual']
    
    # Filtra colunas que realmente existem
    cols_home = [c for c in cols_home if c in df_clean.columns]
    cols_away = [c for c in cols_away if c in df_clean.columns]

    home_stats = df_clean[cols_home].rename(columns={
        'HomeTeam':'Team', 'FTHG':'Goals', 'FTAG':'Conceded', 
        'HS':'Shots', 'HST':'SoT', 'HC':'Corners', 'Home_xG':'xG', 'Home_Points_Actual':'Points'
    })
    
    away_stats = df_clean[cols_away].rename(columns={
        'AwayTeam':'Team', 'FTAG':'Goals', 'FTHG':'Conceded', 
        'AS':'Shots', 'AST':'SoT', 'AC':'Corners', 'Away_xG':'xG', 'Away_Points_Actual':'Points'
    })
    
    all_stats = pd.concat([home_stats, away_stats]).sort_values(['Team', 'Date'])
    
    metrics = ['Points', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners', 'xG']
    
    # Calcular médias móveis (com shift para não ver o futuro)
    for m in metrics:
        if m in all_stats.columns:
            all_stats[f'Avg_{m}'] = all_stats.groupby('Team')[m].transform(
                lambda x: x.shift(1).rolling(window=window, min_periods=3).mean()
            )
        else:
            all_stats[f'Avg_{m}'] = 0

    # --- CORREÇÃO DO ERRO ---
    # Antes do merge, REMOVEMOS as colunas originais que vão colidir ou causar Leakage
    # Isto impede o erro "Columns must be same length" e impede que o modelo veja o xG do jogo atual
    cols_to_ban = ['Home_xG', 'Away_xG', 'Home_Points_Actual', 'Away_Points_Actual']
    df_clean = df_clean.drop(columns=[c for c in cols_to_ban if c in df_clean.columns])

    # Merge das médias (Agora seguro, sem duplicados)
    df_clean = df_clean.merge(all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
                  left_on=['Date', 'HomeTeam'], right_on=['Date', 'Team'], how='left')
    df_clean = df_clean.drop(columns=['Team']).rename(columns={f'Avg_{m}': f'Home_{m}' for m in metrics})
    
    df_clean = df_clean.merge(all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
                  left_on=['Date', 'AwayTeam'], right_on=['Date', 'Team'], how='left')
    df_clean = df_clean.drop(columns=['Team']).rename(columns={f'Avg_{m}': f'Away_{m}' for m in metrics})

    # 4. ODDS
    if 'B365H' in df_clean.columns:
        df_clean['Prob_Home'] = 1 / df_clean['B365H']
        df_clean['Prob_Draw'] = 1 / df_clean['B365D']
        df_clean['Prob_Away'] = 1 / df_clean['B365A']
    
    # Preencher vazios nas features novas
    features_cols = [c for c in df_clean.columns if 'Home_' in c or 'Away_' in c]
    # Remover duplicados da lista se houver (segurança extra)
    features_cols = list(set(features_cols))
    
    df_clean[features_cols] = df_clean[features_cols].fillna(0)
    
    # Limpeza final (apenas linhas sem Odd ou Resultado)
    if 'Prob_Home' in df_clean.columns:
        before_drop = len(df_clean)
        df_clean = df_clean.dropna(subset=['Prob_Home', 'FTR'])
        print(f"📉 Removidos {before_drop - len(df_clean)} jogos sem Odds/Resultado.")

    print(f"✅ FINAL: {len(df_clean)} jogos prontos para treino.")
    return df_clean, elo_dict

# Executar a nova preparação
df_processed, elo_tracker = prepare_features_corrected(df)

## 3. Preparação e Treino do Modelo
Treino Intensivo: Grid Search (Hyperparameter Tuning) Aqui é onde "apertamos" o modelo. Vamos testar várias combinações. Nota: Isto pode demorar 2 ou 3 minutos a correr.

In [ ]:
# CÉLULA DE TREINO (CORRIGIDA)
MODEL_FILE = 'model_xgboost_clean.pkl'
ENCODER_FILE = 'label_encoder.pkl'

# VERIFICAÇÃO DE SEGURANÇA
print(f"🛠️ Dados disponíveis para treino: {len(df_processed)}")
if len(df_processed) < 100:
    raise ValueError("❌ ERRO CRÍTICO: Tens menos de 100 jogos. O treino vai falhar. Verifica os filtros de data e o ficheiro CSV.")


# 1. Definir Features (REMOVER NOMES DAS EQUIPAS)
cols_candidates = ['HomeElo', 'AwayElo', 'EloDiff', 'Prob_Home', 'Prob_Draw', 'Prob_Away'] + \
                  [c for c in df_processed.columns if 'Home_' in c or 'Away_' in c]

# Filtro de Segurança: Remover Nomes e Datas das features
features = [f for f in cols_candidates if f in df_processed.columns]
features = [f for f in features if f not in ['HomeTeam', 'AwayTeam', 'Date', 'Season', 'Referee']]

print(f"🚀 Features finais ({len(features)}): {features}")

le = LabelEncoder()
df_processed['Target'] = le.fit_transform(df_processed['FTR'])

# Split
split = int(len(df_processed) * 0.90)
train = df_processed.iloc[:split]
test = df_processed.iloc[split:]
X_train, y_train = train[features], train['Target']
X_test, y_test = test[features], test['Target']

# Grid Search (Treino)
print("⚠️ A iniciar treino sem 'nomes' (evita overfitting)...")

xgb_model = xgb.XGBClassifier(random_state=42, objective='multi:softprob', eval_metric='mlogloss')
param_grid = {
    'n_estimators': [150, 200],
    'max_depth': [3, 4],
    'learning_rate': [0.03, 0.05],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3]
}

tscv = TimeSeriesSplit(n_splits=3)
grid = GridSearchCV(xgb_model, param_grid, cv=tscv, scoring='accuracy', verbose=1)
grid.fit(X_train, y_train)

best_params = grid.best_params_
print(f"✅ Melhores parâmetros: {best_params}")

# Treino Final com Pesos
weights = np.ones(len(y_train))
draw_idx = le.transform(['D'])[0]
weights[y_train == draw_idx] = 1.3

print("⚖️ A treinar modelo final...")
model_final = xgb.XGBClassifier(**best_params, random_state=42, objective='multi:softprob')
model_final.fit(X_train, y_train, sample_weight=weights)

joblib.dump(model_final, MODEL_FILE)
joblib.dump(le, ENCODER_FILE)
print("💾 Modelo salvo.")

### Matriz de Confusão e accuracy
Vamos ver visualmente onde o modelo erra.
* Eixo Y: O que realmente aconteceu.
* Eixo X: O que o modelo previu.

In [ ]:
# Avaliação
preds = model_final.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"🎯 Accuracy Final (com xG): {acc:.2%}")

# Ver importância das features
importances = pd.Series(model_final.feature_importances_, index=features).sort_values(ascending=False)
print("\nTop 5 Fatores mais importantes:")
print(importances.head(5))

In [ ]:
def diagnose_model(df, model, features, target_col='Target'):
    print("--- 🕵️‍♂️ RELATÓRIO DE DIAGNÓSTICO ---")
    
    # 1. VERIFICAR DATA LEAKAGE (A mais importante!)
    # Vamos ver a correlação de TODAS as features com o Target (Resultado)
    # Se alguma feature tiver correlação > 0.8, é provável que seja leakage (spoiler do resultado)
    print("\n🔍 1. Análise de Data Leakage (Correlações Suspeitas)")
    df_corr = df[features + [target_col]].copy()
    corr_matrix = df_corr.corr()
    target_corr = corr_matrix[target_col].abs().sort_values(ascending=False).drop(target_col)
    
    top_suspects = target_corr.head(10)
    print(top_suspects)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_suspects.values, y=top_suspects.index, palette='Reds_r')
    plt.title("Top Correlações com o Resultado (Atenção se > 0.6)")
    plt.axvline(x=0.6, color='black', linestyle='--', label='Zona de Perigo')
    plt.legend()
    plt.show()

    # 2. MATRIZ DE CONFUSÃO VISUAL
    print("\n🎨 2. Performance Real (Matriz de Confusão)")
    # Split manual rápido para teste (usando o mesmo split do teu código)
    split = int(len(df) * 0.90)
    X_test = df.iloc[split:][features]
    y_test = df.iloc[split:][target_col]
    
    preds = model.predict(X_test)
    
    # Mapear de volta para nomes (0,1,2 -> A, D, H) se usaste LabelEncoder
    # Assumindo ordem alfabética típica do LabelEncoder: 0=Away, 1=Draw, 2=Home
    labels = ['Away', 'Draw', 'Home']
    
    conf_mat = confusion_matrix(y_test, preds)
    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Previsão do Modelo')
    plt.ylabel('O que Realmente Aconteceu')
    plt.title('Onde é que o modelo está a errar?')
    plt.show()

    # 3. CHECAR DADOS HISTÓRICOS (ELO e xG)
    print("\n📈 3. Verificação Visual de Dados (Elo e xG)")
    teams_check = ['Aston Villa', 'Arsenal']  # coloca aqui as equipas
    colors = ['blue', 'red']  # podes mudar as cores se quiseres

    plt.figure(figsize=(12, 4))

    for team_check, color in zip(teams_check, colors):
        # Filtrar jogos da equipa
        df_team = df[(df['HomeTeam'] == team_check) | (df['AwayTeam'] == team_check)].sort_values('Date')

        # Escolher o Elo conforme casa/fora
        df_team['MyElo'] = np.where(
            df_team['HomeTeam'] == team_check,
            df_team['HomeElo'],
            df_team['AwayElo']
        )

        # Plot
        plt.plot(df_team['Date'], df_team['MyElo'], label=f'Elo {team_check}', color=color)

    plt.title("Evolução do Elo das Equipas (deve variar suavemente)")
    plt.grid(True)
    plt.legend()
    plt.show()

# Corre o diagnóstico
diagnose_model(df_processed, model_final, features)

# --- DEBUG ESPECÍFICO DO JOGO VILLA VS ARSENAL ---
def inspect_prediction_input(home, away):
    print(f"\n🔬 RAIO-X: {home} vs {away}")
    
    # Tenta encontrar a última linha de dados usada
    try:
        h_row = df_processed[df_processed['HomeTeam'] == home].iloc[-1]
        a_row = df_processed[df_processed['AwayTeam'] == away].iloc[-1]
        
        print(f"\nDados crus extraídos do histórico para {home}:")
        cols_to_show = ['Date', 'HomeElo', 'Home_xG', 'Home_Goals', 'Home_Shots']
        # Filtra colunas que existem
        cols_exist = [c for c in cols_to_show if c in df_processed.columns]
        print(h_row[cols_exist])
        
        print(f"\nDados crus extraídos do histórico para {away}:")
        print(a_row[cols_exist])
        
        # Verificar se há zeros suspeitos (ex: xG = 0.0 é estranho numa média)
        if h_row.get('Home_xG', 0) == 0 or a_row.get('Away_xG', 0) == 0:
            print("\n⚠️ ALERTA: Foi detetado xG = 0. O Scraper pode ter falhado para esta equipa!")
            
    except Exception as e:
        print(f"Erro ao inspecionar: {e}")

inspect_prediction_input('Aston Villa', 'Arsenal')

## 4. Aplicação na "Vida Real"
Aqui está a função final. Ela usa o dicionário `current_elo` (que contém os valores mais recentes após o último jogo do dataset) para fazer previsões sobre jogos futuros.

In [ ]:
def predict_smart(home, away, odd_h, odd_d, odd_a):
    # --- 1. CARREGAMENTO DO MODELO ---
    # Tenta usar o modelo que está na memória (model_final).
    global model_final, le # Garante que acedemos às variáveis globais se existirem
    
    try:
        model = model_final
    except NameError:
        print("⚠️ Modelo não está na memória. A carregar do disco...")
        model = joblib.load('model_xgboost_xg.pkl') # <--- O NOME CORRETO É ESTE
        
    # Verificar se o LabelEncoder (le) existe, senão carregar
    try:
        encoder = le
    except NameError:
        encoder = joblib.load('label_encoder.pkl')

    # --- 2. PREPARAR DADOS ---
    # (Assume que elo_tracker e df_processed estão em memória)
    h_elo = elo_tracker.get(home, 1500)
    a_elo = elo_tracker.get(away, 1500)
    
    input_data = {
        'HomeElo': h_elo, 'AwayElo': a_elo, 'EloDiff': h_elo - a_elo,
        'Prob_Home': 1/odd_h, 'Prob_Draw': 1/odd_d, 'Prob_Away': 1/odd_a
    }
    
    # Preencher stats históricas (xG, Remates, etc.)
    # Se a equipa subiu de divisão agora e não tem histórico, usamos zeros (segurança)
    try:
        h_row = df_processed[df_processed['HomeTeam'] == home].iloc[-1]
    except IndexError:
        h_row = pd.Series(0, index=df_processed.columns)
        
    try:
        a_row = df_processed[df_processed['AwayTeam'] == away].iloc[-1]
    except IndexError:
        a_row = pd.Series(0, index=df_processed.columns)
    
    # Preencher as features necessárias
    for feat in features:
        if feat not in input_data:
            if 'Home_' in feat: input_data[feat] = h_row.get(feat, 0)
            elif 'Away_' in feat: input_data[feat] = a_row.get(feat, 0)

    X_input = pd.DataFrame([input_data])[features]
    
    # --- 3. PREVISÃO ---
    probs = model.predict_proba(X_input)[0]
    
    # Mapear probabilidades corretamente usando o encoder
    # O encoder sabe que 0=Away, 1=Draw, 2=Home (ou a ordem alfabética 'A', 'D', 'H')
    class_order = encoder.classes_
    prob_dict = {class_label: prob for class_label, prob in zip(class_order, probs)}
    
    p_home = prob_dict.get('H', 0)
    p_draw = prob_dict.get('D', 0)
    p_away = prob_dict.get('A', 0)
    
    print(f"\n🧠 Análise: {home} vs {away}")
    print(f"   Probabilidades: Casa {p_home:.1%} | Empate {p_draw:.1%} | Fora {p_away:.1%}")
    print(f"   (xG Médio Recente: {home} {input_data.get('Home_xG',0):.2f} vs {away} {input_data.get('Away_xG',0):.2f})")
    
    # --- 4. VEREDICTO ---
    prediction = "Inconclusivo"
    
    if p_home > 0.45:
        prediction = f"Vitória {home}"
    elif p_away > 0.45:
        prediction = f"Vitória {away}"
    elif p_draw > 0.28: 
        prediction = "EMPATE (Risco calculado)"
    else:
        max_prob = max(p_home, p_draw, p_away)
        if max_prob == p_home: prediction = f"Tendência {home}"
        elif max_prob == p_away: prediction = f"Tendência {away}"
        else: prediction = "Tendência Empate"

    print(f"   >> Veredicto IA: {prediction}")

# Testa com o jogo que querias
predict_smart('Aston Villa', 'Arsenal', 4.05, 3.45, 1.84)